## Forecasting Gas Prices With Machine Learning Methods

In this Notebook, I extract data on gas prices from a CSV file and try to forecast the data from up to one year past the latest date given in the dataset.

In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.linear_model import Ridge

In [2]:
csv_path = Path("downloads/Nat_Gas.csv")
df = pd.read_csv(csv_path,low_memory = False)

In [3]:
df["Dates"] = pd.to_datetime(df["Dates"])
df = df.sort_values("Dates")

start_date = df["Dates"].min()
df["t"] = (df["Dates"].dt.year - start_date.year) * 12 + \
          (df["Dates"].dt.month - start_date.month)

y = df["Prices"]

/var/folders/tz/wrp91tr15mz13_6kzs5t_vsw0000gn/T/ipykernel_41848/3652016700.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Dates"] = pd.to_datetime(df["Dates"])


In [5]:
def make_fourier_features(t, period=12, K=2):
    features = {}
    for k in range(1, K + 1):
        features[f"sin_{k}"] = np.sin(2 * np.pi * k * t / period)
        features[f"cos_{k}"] = np.cos(2 * np.pi * k * t / period)
    return pd.DataFrame(features)

K = 2  # number of harmonics
fourier = make_fourier_features(df["t"], period=12, K=K)

X = pd.concat([df[["t"]], fourier], axis=1)



In [6]:
model = Ridge(alpha=1.0)
model.fit(X, y)

,alpha,1.0
,fit_intercept,True
,copy_X,True
,max_iter,None
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,None


In [13]:
def estimate_gas_price(date_str: str) -> float:
    """
    Estimate the gas price for a given date.
    Date must be within historical range or up to 12 months in the future.
    """
    date = pd.to_datetime(date_str)

    last_date = ts.index[-1]

    # If date is in the past (or observed range)
    if date <= last_date:
        # Get model-smoothed estimate
        prediction = results.get_prediction(start=date, end=date)
        return float(prediction.predicted_mean.iloc[0])

    # If date is in the future (max 12 months)
    months_ahead = (date.year - last_date.year) * 12 + (date.month - last_date.month)

    if months_ahead > 12:
        raise ValueError("Forecast horizon exceeds 12 months.")

    forecast = results.get_forecast(steps=months_ahead)
    return float(forecast.predicted_mean.iloc[-1])


In [19]:
estimate_gas_price("2019-02-01")

KeyError: 'The `start` argument could not be matched to a location related to the index of the data.'